# 어프라이어리 알고리즘으로 연관 분석하기

## 11.1 Association analysis

- Apriori
    - Pros: Easy to code up
    - Cons: May be slow on large datasets
    - Works with: Numeric values, nominal values

- Sparse Matrix : 인스턴스에 존재하는 아이템만 체크한 matrix
- 지지도(support)
    - 이 아이템 집합이 포함된 데이터 집합의 비율로 정의한다. 
- 신뢰도(confidence)
    - "기저귀 -> 와인" 처럼 연관 규칙으로 정의한다. 
    - 이 규칙에 대한 신뢰도는 지지도({기저귀, 와인}/)지지도({기저귀}) 가 된다.
- 지지도와 신뢰도는 연관 분석의 성공 여부를 측정할 수 있는 방법이다. 우리가 0.8보다 큰 지지도를 가지는 아이템들의 모든 집합을 찾는다고 가정해 보자. 이를 어떻게 찾을 것인가?

어프라이어리 이론은 연관 규칙을 얻기 위해 해야만 하는 계산의 회수를 줄일 수 있게 해 줄 것이다.

## 11.2 The Apriori principle

General approach to the Apriori algorithm
1. Collect: 
    - Any method.
2. Prepare: 
    - Any data type will work as we’re storing sets.
3. Analyze: 
    - Any method.
4. Train: 
    - Use the Apriori algorithm to find frequent itemsets.
5. Test: 
    - Doesn’t apply.
6. Use: 
    - This will be used to find frequent itemsets and association rules between items.

- Apriori

A priori means “from before” in Latin. When defining a problem, it’s common to state
prior knowledge, or assumptions. This is written as “a priori.” In Bayesian statistics,
it’s common to make inferences conditional upon this a priori knowledge. A priori
knowledge can come from domain knowledge, previous measurements, and so on.

어 프라이어리는 라틴어로 '이전으로부터' 라는 의미이다. 이는 문제를 정의할 때, 일반적으로 이전 지식의 상태 또는 가정들을 의미하게 된다. 이를 "어 프라이어리"로 불렀으며, 베이스 통계학에서 이전지식을 기반으로 추론 조건을 생성하는 데 일반적으로 사용된다. 이전 지식은 해당 지식의 분야, 이전에 측정한 결과 등에서 가져올 수 있다.

대우가 이용되네, $p->q <=(대우)=> not \ q-> not \ p$

## 11.3 Finding frequent itemsets with the Apriori algorithm

### 11.3.1 Generating candidate itemsets

we’ll need to create a few helper functions.

#### Listing 11.1 Apriori algorithm helper functions

In [1]:
from numpy import *

def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]] # 함께 사간 item 묶음!! 

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if [item] not in C1: # [],list 만든거지
                C1.append([item])
                
    C1.sort()
    return map(frozenset, C1)#use frozen set so we
                            #can use it as a key in a dict
    # frozenset은, 집합의 개수를 couniting 하기 위해서 
    # frozenset 따로 수정이 안되네. 내용이 바뀌지 않는 set(집합)이다!! 

def scanD(D, Ck, minSupport): # Ck = C1 C2 C3 될 거지!! 
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid): # 부분집합이라면
                if not ssCnt.has_key(can): 
                    ssCnt[can]=1
                else: 
                    ssCnt[can] += 1
                # counting 증가시키네!! 1상품 몇번, 2상품 몇번..
                
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport: # 나오는 비율이 minSupport 보다 크면 보고 아니면 안보고 
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData





In [3]:
pwd

u'C:\\pydev\\Machine_Learning\\Machine_Learning_in_Action_py2\\Ch11'

In [5]:
import apriori

In [6]:
dataSet = apriori.loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [7]:
C1 = apriori.createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [9]:
D = map(set, dataSet) # dataSet을 집합으로 만듬
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [11]:
L1, suppData0 = apriori.scanD(D, C1, 0.5) # minimum support = 0.5
L1 # 4는 없지 이제 4가 포함된 것은 보지 않을 것이다!! 

[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})]

In [13]:
suppData0 # 각 item에 대한 지지도를 보여주고 있다!!

{frozenset({4}): 0.25,
 frozenset({5}): 0.75,
 frozenset({2}): 0.75,
 frozenset({3}): 0.75,
 frozenset({1}): 0.5}

### 11.3.2 Putting together the full Apriori algorithm

#### Listing 11.2 The Apriori algorithm

In [15]:
def aprioriGen(Lk, k): #creates Ck # helper function 
    # Lk = [f{1},f{2},f{3},f{4},f{5}]
    # k = 2 가 들어오지!!  
    
    retList = []
    lenLk = len(Lk) # 4
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            # 공집합부터 보게 되네, list(Lk[0])[:0]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = map(set, dataSet)
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1] # L1은 살아남은 녀석들! 
    k = 2 # 2개로 구정된 집합들을 찾겠다! 1,2,5를 가지고 구성하지!! 
    while (len(L[k-2]) > 0): # L[0] L[1] L[2] L[3]을 봄!! 
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [21]:
list(L1[0])

[1]

In [20]:
list(L1[0])[:0]

[]

- 합집합이지!!

In [24]:
L1[0] | L1[1]

frozenset({1, 3})

In [28]:
import apriori

In [29]:
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [33]:
L, suppData = apriori.apriori(dataSet)
L # 봐야하는 집합만 출력해줌!! 

[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
 [frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3}), frozenset({3, 5})],
 [frozenset({2, 3, 5})],
 []]

$2^5-1=31$ 보다 작은 9가지 경우만 보면 된다!! 

In [32]:
suppData

{frozenset({5}): 0.75,
 frozenset({3}): 0.75,
 frozenset({2, 3, 5}): 0.5,
 frozenset({1, 2}): 0.25,
 frozenset({1, 5}): 0.25,
 frozenset({3, 5}): 0.5,
 frozenset({4}): 0.25,
 frozenset({2, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({1}): 0.5,
 frozenset({1, 3}): 0.5,
 frozenset({2}): 0.75}

### 11.4 Mining association rules from frequent item sets
빈발 아이템 집합으로 연관규칙 마이닝하기

- 두유->상추라는 것이 상추->두유라는 것을 의미하지는 않는다(선핸조건 $->$ 결과)
- 신뢰도(confidence) = $support(P|H) \ / \ support(P)$    ["|"은 합집합을 의미한다!!]

In [34]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print freqSet-conseq,'-->',conseq,'conf:',conf
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)
            # 재귀 쓰네!! 
            

In [35]:
L, suppData = apriori.apriori(dataSet, minSupport=0.5)

In [36]:
L

[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
 [frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3}), frozenset({3, 5})],
 [frozenset({2, 3, 5})],
 []]

In [37]:
suppData

{frozenset({5}): 0.75,
 frozenset({3}): 0.75,
 frozenset({2, 3, 5}): 0.5,
 frozenset({1, 2}): 0.25,
 frozenset({1, 5}): 0.25,
 frozenset({3, 5}): 0.5,
 frozenset({4}): 0.25,
 frozenset({2, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({1}): 0.5,
 frozenset({1, 3}): 0.5,
 frozenset({2}): 0.75}

In [41]:
rules = apriori.generateRules(L, suppData, minConf=0.8) # 신뢰도 80% 되어야 확실한 관계라고 할 수 있다!!

frozenset([1]) --> frozenset([3]) conf: 1.0
frozenset([5]) --> frozenset([2]) conf: 1.0
frozenset([2]) --> frozenset([5]) conf: 1.0


In [43]:
rules = apriori.generateRules(L, suppData, minConf=0.5) # 신뢰도 50% 되어야 확실한 관계라고 할 수 있다!!

frozenset([3]) --> frozenset([1]) conf: 0.666666666667
frozenset([1]) --> frozenset([3]) conf: 1.0
frozenset([5]) --> frozenset([2]) conf: 1.0
frozenset([2]) --> frozenset([5]) conf: 1.0
frozenset([3]) --> frozenset([2]) conf: 0.666666666667
frozenset([2]) --> frozenset([3]) conf: 0.666666666667
frozenset([5]) --> frozenset([3]) conf: 0.666666666667
frozenset([3]) --> frozenset([5]) conf: 0.666666666667
frozenset([5]) --> frozenset([2, 3]) conf: 0.666666666667
frozenset([3]) --> frozenset([2, 5]) conf: 0.666666666667
frozenset([2]) --> frozenset([3, 5]) conf: 0.666666666667


- 인과관계, 연관관계는 다르지!! 